<a href="https://colab.research.google.com/github/Tom-Jung/IO_with_python/blob/main/Ch.%20%EB%B6%80%EB%A1%9D_%EB%B6%80%EB%AC%B8%ED%86%B5%ED%95%A9_%EC%A7%80%EC%97%ADIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

In [150]:
np.set_printoptions(precision=3,suppress=True) # 결과를 소수 3자리까지만 보여주는 옵션

- 2015년 지역산업연관표(33부문, 17개 지역)를 IO표를 3부문, 3지역 IO표로 통합하고 이를 통해 설명하기로 함

- 작업방법은 전국산업연관표와 동일함

In [151]:
url_RIO_33 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Regional_Matrix.csv"  # 33부문의 지역IO 불러오기
RIO_33 = pd.read_csv(url_RIO_33)

In [152]:
RRIO_33 = RIO_33.to_numpy()
RI_33 = RRIO_33[2:,3:].astype(float)

In [153]:
Cd_for_R = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/code_regional_io_3regions_3sectors.csv"  # 코드매치 파일
Cdr = pd.read_csv(Cd_for_R)
Cdr.head()

,권역,지역,33부문,부문명,3부문,old_33부문,new_3부문및지역
0,수도권,서울,A,농림수산품,농산품,1,1
1,수도권,서울,B,광산품,공산품,2,2
2,수도권,서울,C01,음식료품,공산품,3,2
3,수도권,서울,C02,섬유 및 가죽제품,공산품,4,2
4,수도권,서울,C03,"목재 및 종이, 인쇄",공산품,5,2


## 국산거래표 통합하기

In [154]:
Cdr_for_itr = Cdr[["old_33부문", "new_3부문및지역"]]
Cd_itr = Cdr_for_itr.to_numpy()
Cd_itr.shape

(594, 2)

In [155]:
Row = Cd_itr[:,1]
Col = Cd_itr[:,0]

In [156]:
Mtx = np.zeros((12,594))       # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽지 않으므로 반복문을 통해 3x33 임시코드 만들기
for i, j in zip(Row,Col):
  Mtx[i-1,j-1] = 1
print(Mtx)
Code_9 = Mtx[:9,:561]

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]


In [157]:
Code_9.shape

(9, 561)

In [158]:
Dom_tot = (Code_9@RI_33[:561,:561])@Code_9.T # 국산중간투입(수요)을 권역별 3부문으로 만들기
print('3부문_3지역 국산내생거래표', '\n', Dom_tot)

3부문_3지역 국산내생거래표 
 [[2.334e+05 3.116e+06 9.829e+05 4.540e+04 3.636e+05 1.138e+05 5.824e+04
  1.249e+06 1.066e+05]
 [1.870e+06 1.121e+08 7.111e+07 7.871e+05 3.591e+07 1.391e+07 1.095e+06
  3.690e+07 1.293e+07]
 [7.807e+05 7.861e+07 3.016e+08 8.469e+05 2.655e+07 3.115e+07 1.151e+06
  2.829e+07 2.839e+07]
 [7.487e+04 1.356e+06 2.089e+06 9.460e+05 7.314e+06 2.210e+06 1.441e+05
  1.845e+06 4.930e+05]
 [2.502e+05 4.308e+07 3.842e+07 3.509e+06 1.580e+08 3.975e+07 1.168e+06
  3.614e+07 1.269e+07]
 [4.225e+04 3.582e+06 1.538e+07 1.898e+06 5.928e+07 7.637e+07 1.609e+05
  3.375e+06 3.664e+06]
 [9.182e+04 2.244e+06 2.805e+06 1.415e+05 1.475e+06 8.411e+05 1.128e+06
  1.186e+07 1.532e+06]
 [4.700e+05 4.631e+07 3.964e+07 1.432e+06 4.100e+07 1.519e+07 5.921e+06
  8.764e+07 3.129e+07]
 [3.748e+04 3.324e+06 1.683e+07 9.508e+04 2.591e+06 3.270e+06 1.856e+06
  4.052e+07 5.581e+07]]


### 수입거래표 통합하기

In [159]:
Cd_im_rw = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/code.csv"  # 코드매치 파일
Cd_i_rw = pd.read_csv(Cd_im_rw)
Cd_ir_itr = Cd_i_rw.to_numpy().reshape(33,2)
Row_i = Cd_ir_itr[:,1]
Col_i = Cd_ir_itr[:,0]
Col_i

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [160]:
Mtx_i = np.zeros((3,33))       # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽지 않으므로 반복문을 통해 3x33 임시코드 만들기
for i, j in zip(Row_i,Col_i):
  Mtx_i[i-1,j-1] = 1
Code_3 = Mtx_i
Code_3.T.shape

(33, 3)

In [161]:
Imp_row = (Code_3@RI_33[562:595,0:561])@Code_9.T # 3X18 수입거래표
print('3부문_3지역 수입내생거래표', '\n', Imp_row)

3부문_3지역 수입내생거래표 
 [[4.239e+04 2.882e+06 3.991e+05 1.310e+05 1.995e+06 1.937e+05 1.754e+05
  3.342e+06 1.558e+05]
 [1.171e+05 1.002e+08 4.105e+07 4.540e+05 1.295e+08 1.836e+07 5.861e+05
  9.813e+07 1.694e+07]
 [2.642e+04 5.651e+06 2.549e+07 2.137e+05 5.952e+06 1.259e+07 1.346e+05
  4.338e+06 8.158e+06]]


### 최종수요 통합하기

In [162]:
Cd_fd = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/code_regional_io_fd.csv"  # 코드매치 파일
Cd_fd1 = pd.read_csv(Cd_fd)
Cd_fd1.head()
# cd_fd2 = cd_fd1.to_numpy()
# row_fd = cd_fd2[:,1]
# col_fd = cd_fd2[:,0]
# row_fd

,권역,항목코드,항목,old_33부문,new_3부문및지역
0,서울,9111,민간소비지출\n(가계 및 가계봉사\n비영리단체),1,1
1,서울,9112,정부소비지출,2,1
2,서울,9121,민간고정\n자본형성,3,2
3,서울,9122,정부고정\n자본형성,4,2
4,서울,9131,재고증감,5,2


In [163]:
Cd_fd2 = Cd_fd1[["old_33부문", "new_3부문및지역"]]
Cd_fd2.head()


,old_33부문,new_3부문및지역
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2


In [164]:
Cd_fd3 = Cd_fd2.to_numpy()
Cd_fd3.shape

(119, 2)

In [165]:
Row_fd = Cd_fd3[:,1]
Col_fd = Cd_fd3[:,0]
Col_fd

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119])

In [166]:
Mtx_fd = np.zeros((9,119))       # 7가지 최종수요 항목을 소비, 투자, 수출로 통합
for i, j in zip(Row_fd,Col_fd):
  Mtx_fd[i-1,j-1] = 1
print(Mtx_fd)
Code_fd_9 = Mtx_fd[:9,:119]
print(Code_fd_9.shape)

[[1. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
(9, 119)


In [167]:
Fd_dom = RI_33[:561,562:681].astype(float)  # 국산 최종수요
Fd_imp = RI_33[562:595,562:681].astype(float) # 수입 최종수요
print(Fd_dom.sum())
print(Fd_imp.sum())
print(Fd_dom.shape)

2114693165.0
146759866.0
(561, 119)


In [168]:
Fd_dom_3 = (Code_9@Fd_dom)@Code_fd_9.T # 국산최종수요
Fd_dom_3.shape

(9, 9)

In [169]:
Fd_imp_3 = (Code_3@Fd_imp)@Code_fd_9.T # 수입최종수요
Fd_imp_3.shape

(3, 9)

In [170]:
print(Fd_dom_3.sum())
print(Fd_imp_3.sum())

2114693164.9999998
146759866.0


In [171]:
Fd_col_sm= (RI_33[596,562:681]@Code_fd_9.T).reshape([1,9]) # 국산 및 수입최종수요 열합
Fd_col_sm.sum()

2261453031.0

In [172]:
Fd_each = np.concatenate((Fd_dom_3, Fd_imp_3,Fd_col_sm, np.zeros((6,9))), axis=0) # 각 지역의 최종수요와 아래에 0 붙이기
print(Fd_each)

[[ 1.167e+06  1.027e+05  7.995e+04  1.420e+05  1.454e+04  0.000e+00
   1.654e+05  2.002e+04  0.000e+00]
 [ 2.903e+07  2.085e+07  2.146e+08  9.195e+06  6.175e+06  0.000e+00
   7.340e+06  7.726e+06  0.000e+00]
 [ 3.874e+08  1.641e+08  6.726e+07  3.299e+07  1.256e+07  0.000e+00
   3.095e+07  1.108e+07  0.000e+00]
 [ 2.558e+06  5.288e+04  0.000e+00  2.663e+06  5.903e+05  2.697e+05
   8.104e+05  4.371e+04  0.000e+00]
 [ 1.992e+07  1.454e+07  0.000e+00  1.928e+07  1.120e+07  2.264e+08
   7.388e+06  6.960e+06  0.000e+00]
 [ 1.098e+07  2.334e+06  0.000e+00  1.938e+08  7.650e+07  2.590e+07
   4.905e+06  1.221e+06  0.000e+00]
 [ 3.334e+06  7.310e+04  0.000e+00  1.128e+06  6.231e+04  0.000e+00
   2.182e+06  7.134e+05  3.375e+05]
 [ 1.491e+07  8.830e+06  0.000e+00  6.780e+06  3.720e+06  0.000e+00
   1.529e+07  4.848e+06  1.676e+08]
 [ 1.489e+07  3.562e+06  0.000e+00  5.371e+06  5.790e+05  0.000e+00
   1.459e+08  7.102e+07  1.236e+07]
 [ 1.457e+06  9.226e+03  0.000e+00  7.629e+05 -1.711e+04  0.000e

In [173]:
Fd_tot_sm = Fd_each.sum(axis=1).reshape([19,1])

##부가가치 통합하기

In [174]:
Vact = RI_33[596:,:561]  # 부가가치 및 산출액
Vact

array([[  28585.,    3634., 1695333., ...,  612973.,  310224.,   46129.],
       [  11062.,    1168.,  286576., ...,  277510.,  186723.,       0.],
       [  41299.,    1902.,   84735., ...,  169252.,   57232.,       0.],
       ...,
       [   1913.,      41.,  103553., ...,  135051.,   27509.,       0.],
       [  59846.,    3768.,  594194., ...,  736637.,  296313.,       0.],
       [  88431.,    7402., 2289527., ..., 1349610.,  606537.,   46129.]])

In [175]:
Vact_3 = Vact@Code_9.T  # 3부문 부가가치
Vact_3.sum()

9304574828.0

In [176]:
Dom_tot.shape

(9, 9)

In [177]:
Imp_row.shape

(3, 9)

In [178]:
Nae_tot = np.concatenate((Dom_tot, Imp_row, Vact_3)) # 중간투입(수요)와 부가가치 및 산출액 합치기
Nae_tot.shape

(19, 9)

In [179]:
Nae_tot.shape

(19, 9)

In [180]:
Tot_row_sm= Nae_tot.sum(axis=1).reshape([19,1])
Tot_row_sm.sum()

11500686239.99995

In [181]:
Tot_dm = Tot_row_sm + Fd_tot_sm  # 총수요 = 중간수요계+최종수요계

In [182]:
Tot_rg = np.concatenate((Nae_tot,Tot_row_sm,Fd_each, Fd_tot_sm,Tot_dm), axis=1)
print('3부문_3지역산업연관표', '\n', (Tot_rg/1000000).round(0))

3부문_3지역산업연관표 
 [[   0.    3.    1.    0.    0.    0.    0.    1.    0.    6.    1.    0.
     0.    0.    0.    0.    0.    0.    0.    2.    8.]
 [   2.  112.   71.    1.   36.   14.    1.   37.   13.  287.   29.   21.
   215.    9.    6.    0.    7.    8.    0.  295.  582.]
 [   1.   79.  302.    1.   27.   31.    1.   28.   28.  497.  387.  164.
    67.   33.   13.    0.   31.   11.    0.  706. 1204.]
 [   0.    1.    2.    1.    7.    2.    0.    2.    0.   16.    3.    0.
     0.    3.    1.    0.    1.    0.    0.    7.   23.]
 [   0.   43.   38.    4.  158.   40.    1.   36.   13.  333.   20.   15.
     0.   19.   11.  226.    7.    7.    0.  306.  639.]
 [   0.    4.   15.    2.   59.   76.    0.    3.    4.  164.   11.    2.
     0.  194.   77.   26.    5.    1.    0.  316.  479.]
 [   0.    2.    3.    0.    1.    1.    1.   12.    2.   22.    3.    0.
     0.    1.    0.    0.    2.    1.    0.    8.   30.]
 [   0.   46.   40.    1.   41.   15.    6.   88.   31.  269.   15. 

In [183]:
Mt_df = pd.DataFrame((Tot_rg/1000000).round(0), columns = [['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권','지역중간수요계',
                                     '수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권','지역최종수요계','지역총수요계'],

                                      ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품', '서비스', '제품중간수요계',
                                       '소비', '투자', '수출', '소비', '투자', '수출', '소비', '투자', '수출','제품최종수요계','제품총수요계']],

                                    index=[['수도권', '수도권', '수도권', '동부권', '동부권', '동부권','서부권', '서부권', '서부권', '수입','수입', '수입', '중간투입계',
                                            '피보', '영잉', '고소', '생산세f', '부가가치계f', '총투입액f'],
                                           ['농산품', '공산품', '서비스', '농산품', '공산품', '서비스','농산품', '공산품',  '서비스','농산품', '공산품','서비스','중간수요계',
                                            '피용자보수', '영업잉여', '고정자본소모', '생산세', '부가가치계', '총투입액']])
Mt_df.index.names=['지역', '상품']

In [184]:
Mt_df.to_csv('지역산업연관표_3부문_3지역.csv', encoding="utf-8-sig")

In [185]:
Mt_df.index.names=['지역', '상품']
# df_pivot = df_pivot.pivot(index='date', columns='country', values='Population')
# # df_pivot.head
Mt_df


수도권                  동부권                 서부권                \
               농산품    공산품     서비스   농산품    공산품    서비스   농산품    공산품    서비스   
지역     상품                                                                   
수도권    농산품     0.0    3.0     1.0   0.0    0.0    0.0   0.0    1.0    0.0   
       공산품     2.0  112.0    71.0   1.0   36.0   14.0   1.0   37.0   13.0   
       서비스     1.0   79.0   302.0   1.0   27.0   31.0   1.0   28.0   28.0   
동부권    농산품     0.0    1.0     2.0   1.0    7.0    2.0   0.0    2.0    0.0   
       공산품     0.0   43.0    38.0   4.0  158.0   40.0   1.0   36.0   13.0   
       서비스     0.0    4.0    15.0   2.0   59.0   76.0   0.0    3.0    4.0   
서부권    농산품     0.0    2.0     3.0   0.0    1.0    1.0   1.0   12.0    2.0   
       공산품     0.0   46.0    40.0   1.0   41.0   15.0   6.0   88.0   31.0   
       서비스     0.0    3.0    17.0   0.0    3.0    3.0   2.0   41.0   56.0   
수입     농산품     0.0    3.0     0.0   0.0    2.0    0.0   0.0    3.0    0.0   
       공산품     0.0  100.0    41.0   0.0  130.0   18.0   1.0   98.0   17.0   
       서비스     0.0    6.0    25.0   0.0    6.0   13.0   0.0    4.0    8.0   
중간투입계  중간수요계   4.0  403.0   556.0  10.0  470.0  214.0  14.0  354.0  172.0   
피보     피용자보수   0.0   71.0   324.0   2.0   63.0  135.0   3.0   43.0  109.0   
영잉     영업잉여    3.0   52.0   170.0   9.0   45.0   64.0  11.0   43.0   45.0   
고소     고정자본소모  1.0   45.0   104.0   2.0   37.0   48.0   2.0   32.0   38.0   
생산세f   생산세     0.0   11.0    51.0   0.0   23.0   18.0   0.0   19.0   14.0   
부가가치계f 부가가치계   4.0  179.0   648.0  13.0  169.0  265.0  16.0  137.0  206.0   
총투입액f  총투입액    8.0  582.0  1204.0  23.0  639.0  479.0  30.0  491.0  378.0   

              지역중간수요계  ...    수도권           동부권                  서부권         \
              제품중간수요계  ...     투자     수출     소비     투자     수출     소비     투자   
지역     상품              ...                                                    
수도권    농산품        6.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
       공산품      287.0  ...   21.0  215.0    9.0    6.0    0.0    7.0    8.0   
       서비스      497.0  ...  164.0   67.0   33.0   13.0    0.0   31.0   11.0   
동부권    농산품       16.0  ...    0.0    0.0    3.0    1.0    0.0    1.0    0.0   
       공산품      333.0  ...   15.0    0.0   19.0   11.0  226.0    7.0    7.0   
       서비스      164.0  ...    2.0    0.0  194.0   77.0   26.0    5.0    1.0   
서부권    농산품       22.0  ...    0.0    0.0    1.0    0.0    0.0    2.0    1.0   
       공산품      269.0  ...    9.0    0.0    7.0    4.0    0.0   15.0    5.0   
       서비스      124.0  ...    4.0    0.0    5.0    1.0    0.0  146.0   71.0   
수입     농산품        9.0  ...    0.0    0.0    1.0   -0.0    0.0    1.0   -0.0   
       공산품      405.0  ...   29.0    2.0   15.0   12.0    0.0   11.0   13.0   
       서비스       63.0  ...    5.0    0.0    7.0    1.0    0.0    7.0    1.0   
중간투입계  중간수요계   2196.0  ...  248.0  284.0  294.0  124.0  253.0  234.0  118.0   
피보     피용자보수    750.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
영잉     영업잉여     441.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
고소     고정자본소모   310.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
생산세f   생산세      136.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
부가가치계f 부가가치계   1637.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
총투입액f  총투입액    3834.0  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                     지역최종수요계  지역총수요계  
                  수출 제품최종수요계  제품총수요계  
지역     상품                             
수도권    농산품       0.0     2.0     8.0  
       공산품       0.0   295.0   582.0  
       서비스       0.0   706.0  1204.0  
동부권    농산품       0.0     7.0    23.0  
       공산품       0.0   306.0   639.0  
       서비스       0.0   316.0   479.0  
서부권    농산품       0.0     8.0    30.0  
       공산품     168.0   222.0   491.0  
       서비스      12.0   254.0   378.0  
수입     농산품       0.0     3.0    12.0  
       공산품       0.0   109.0   514.0  
       서비스       0.0    35.0    98.